In [ ]:
import mxnet as mx
import numpy as np
import time
from sklearn.cluster import KMeans


In [ ]:
def get_channelwise_clustered(filters, shrink):
    shape = filters.shape
    result=np.zeros(shape)
    n_clusters = int(shape[0] / shrink)

    for channel_idx in range(shape[1]):

        channelwise_filters = filters[:,channel_idx,:,:]
        cw_shape=channelwise_filters.shape

        channelwise_filters_shaped = channelwise_filters.reshape((cw_shape[0], cw_shape[1] * cw_shape[2]))
        estimator = KMeans(n_clusters=n_clusters)
        estimator.fit(channelwise_filters_shaped.asnumpy())

        cw_filter_kmean_indexes = estimator.predict(X=channelwise_filters_shaped.asnumpy())
        cw_filters_quantized = np.array([estimator.cluster_centers_[idx] for idx in cw_filter_kmean_indexes])

        result[:,channel_idx,:,:]=cw_filters_quantized.reshape(cw_shape)

    return mx.nd.array(result)
def get_speedup(in_args,layers):
    original = 0
    clustered = 0

    for layer in layers:
        filter=in_args[layer]
        shape=filter.shape


        for channel in range(shape[1]):
            filters_in_channel = filter[:,channel,:,:]
            nclusters_channel = np.unique(filters_in_channel.asnumpy(),axis=0)
            clustered+=nclusters_channel.shape[0]
            #print nclusters_channel.shape[0]

        original+=shape[0]*shape[1]
    print original
    print clustered

    return float(original)/clustered
def naive_clusternet(sym,args,shrink):
    layers=[]
    for l in sym.get_internals().list_outputs():
        if "weight" in l and "fc" not in l:
            layers.append(l)
            #args[l] = get_quantized(args[l], 8)
            args[l] = get_channelwise_clustered(args[l], shrink)

    return layers

In [ ]:
prefix = "../cnn_models/alexnet/bvlc_alexnet"
epoch = 0
sym, args, auxs = mx.mod.module.load_checkpoint(prefix, epoch)


In [ ]:
layers = naive_clusternet(sym,args,8)
get_speedup(args,layers)

In [ ]:
mod = mx.mod.Module(symbol=sym, context=mx.cpu())
mod.bind(for_training=False, data_shapes=(32,3,224,224))
mod.set_params(args, auxs, allow_missing=True)
mod.save_checkpoint(prefix=prefix+"_clustered8x",epoch=0)

In [ ]:
############## to test if clustering is scucessful
prefix_test = prefix+"_clustered8x"
epoch_test = 0
sym_test, args_test, auxs_test = mx.mod.module.load_checkpoint(prefix_test, epoch_test)
layers_test=[l for l in sym_test.get_internals().list_outputs() if "weight" in l and "fc" not in l]
get_speedup(args_test,layers=layers_test)

In [ ]:
def get_tensor(symbol_input, args, auxs, data_iter):
    mod=mx.mod.Module(symbol=symbol_input, context=mx.cpu(),label_names=None)
    mod.bind(for_training=False, data_shapes=data_iter.provide_data)
    mod.set_params(args, auxs)
    return mod.predict(eval_data=data_iter)


In [ ]:
data_iter=mx.io.NDArrayIter(mx.nd.random_uniform(0,1,shape=(2,3,224,224)), batch_size=2)

In [ ]:
sym_conv1_in=sym.get_internals()['data']
sym_conv1_out=sym.get_internals()['conv1_output']

tensor_conv1_in  = get_tensor(sym_conv1_in,data_iter)
tensor_conv1_out = get_tensor(sym_conv1_out,data_iter)

In [ ]:
tensor_conv1_out.shape

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import AlexNet

In [ ]:
sym_alex= AlexNet.get_symbol(1000)

In [ ]:
for x,y in zip(sym_test.get_internals().list_outputs(), sym_alex.get_internals().list_outputs()):
    if x!=y:
        print "false"
        print x
        print y

In [ ]:
sym_alex.get_internals().list_outputs()

In [ ]:
tensor1=get_tensor(sym_test.get_internals()['prob_output'],args_test, auxs_test, data_iter=data_iter)
tensor2=get_tensor(sym_alex.get_internals()['prob_output'], args_test, auxs_test, data_iter=data_iter)


In [ ]:
np.array_equal(tensor1.asnumpy(), tensor2.asnumpy())